##### Structure of the document
1. Import of experimental data
2. Preprocessing of the data
3. Calculations of new parameters
4. Visualization

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np

pd.options.mode.chained_assignment = None  # Suppress the warning

##### 1. Import experimental data

- Temperature
- Stirring
- Feed of Glucose
- Base & Acid
- dissolved O2
- pH
- air feed
- offgas analysis
- biomass data
- hplc measurements for glucose concentration

In [ ]:
df_T=pd.read_csv('data/fermentation raw data/May 26, 2023_10 58 18 AM_DTU 3-parameterValues_Temperature_Value.csv', sep=';')
df_stir=pd.read_csv('data/fermentation raw data/May 26, 2023_10 58 18 AM_DTU 3-parameterValues_Stirrer_Value.csv', sep=';')
df_feed1=pd.read_csv('data/fermentation raw data/May 26, 2023_10 58 18 AM_DTU 3-parameterValues_Feed 1_Value.csv', sep=';')
df_base=pd.read_csv('data/fermentation raw data/May 26, 2023_10 58 18 AM_DTU 3-parameterValues_Pump - Base_Value.csv', sep=';')
df_acid=pd.read_csv('data/fermentation raw data/May 26, 2023_10 58 18 AM_DTU 3-parameterValues_Pump - Acid_Value.csv', sep=';')
df_pO2=pd.read_csv('data/fermentation raw data/May 26, 2023_10 58 18 AM_DTU 3-parameterValues_pO₂_Value.csv', sep=';')
df_pH=pd.read_csv('data/fermentation raw data/May 26, 2023_10 58 18 AM_DTU 3-parameterValues_pH_Value.csv', sep=';')
df_air=pd.read_csv('data/fermentation raw data/May 26, 2023_10 58 18 AM_DTU 3-parameterValues_Flow - Air_Value.csv', sep=';')
df_offgas=pd.read_excel('data/fermentation raw data/offgas_data.xlsx')
df_biomass=pd.read_csv('data/fermentation raw data/biomass_OD.csv', sep=';')
df_hplc = pd.read_excel('data/fermentation raw data/HPLC.xlsx')

##### 2. Data Preprocessing

In [ ]:
# group the dataframes -- acid and base will be pre-processed differently
first_para=[df_T, df_stir, df_feed1, df_pO2, df_pH, df_air]
second_para=[df_base, df_acid]

In [ ]:
start = '31-05-2023 10:14:00'
end = '02-06-2023 08:22:00'
start_time = datetime.strptime(start, '%d-%m-%Y %H:%M:%S')
end_time = datetime.strptime(end, '%d-%m-%Y %H:%M:%S')

First parameter group (mean value for reduction of sampling frequency):

In [ ]:
# dataframes are prepared to be merged into one dataframe based on the timestamp
df_merged1=pd.DataFrame()
for df in first_para:
    df_copy=df.copy()
    # convert the timestamp into the datetime format and drop the original timestamp
    df_copy.insert(0, 'DateTime', df_copy['Date'].astype(str)+' '+df_copy['Time'].astype(str))
    df_copy.drop(['Date', 'Time'], axis=1, inplace=True)
    df_copy['DateTime']=pd.to_datetime(df_copy['DateTime'], format='%b %d, %Y %I:%M:%S %p')

    # values are converted to floats
    df_copy[df_copy.columns[1]]=df_copy[df_copy.columns[1]].str.replace(',', '.')
    df_copy[df_copy.columns[1]]=pd.to_numeric(df_copy[df_copy.columns[1]], errors='coerce')

    # reduce sample frequency to 1min which makes the merge easier
    ## mean value of each minute is taken
    df_copy.set_index('DateTime', inplace=True)
    df_resampled = df_copy.resample('1T').mean() # could be changed to sum/first/min/max...
    df_resampled = df_resampled.reset_index()
    
    # dataframe is reduced to the start and end time of the fermentation
    df_se=df_resampled[(df_resampled['DateTime'] > start_time) & (df_resampled['DateTime'] < end_time)]
    df_se = df_se.reset_index(drop=True)
    if df_merged1.empty:
        df_merged1=df_se
    else:
        df_merged1=df_merged1.merge(df_se, on='DateTime', how='inner')

Second parameter group (sum for reduction of sampling frequency):

In [ ]:
# dataframes are prepared to be merged into one dataframe based on the timestamp
# since the feed of base/acid lasts only 5s, data will be lost if we take the first value
## instead we take the sum of the values within that minute and then later divide it by (5/60)
df_merged2=pd.DataFrame()
for df in second_para:
    df_copy=df.copy()
    # convert the timestamp into the datetime format and drop the original timestamp
    df_copy.insert(0, 'DateTime', df_copy['Date'].astype(str)+' '+df_copy['Time'].astype(str))
    df_copy.drop(['Date', 'Time'], axis=1, inplace=True)
    df_copy['DateTime']=pd.to_datetime(df_copy['DateTime'], format='%b %d, %Y %I:%M:%S %p')

    # values are converted to floats
    df_copy[df_copy.columns[1]]=df_copy[df_copy.columns[1]].str.replace(',', '.')
    df_copy[df_copy.columns[1]]=pd.to_numeric(df_copy[df_copy.columns[1]], errors='coerce')

    # reduce sample frequency to 1min which makes the merge easier
    ## mean value of each minute is taken
    df_copy.set_index('DateTime', inplace=True)
    df_resampled = df_copy.resample('1T').sum() # could be changed to sum/mean/first/min/max...
    df_resampled = df_resampled.reset_index()
    
    # dataframe is reduced to the start and end time of the fermentation
    df_se=df_resampled[(df_resampled['DateTime'] > start_time) & (df_resampled['DateTime'] < end_time)]
    df_se = df_se.reset_index(drop=True)
    if df_merged2.empty:
        df_merged2=df_se
    else:
        df_merged2=df_merged2.merge(df_se, on='DateTime', how='inner')

In [ ]:
# merging the 2 dataframes
df_merged3 = df_merged1.copy()
df_merged3 = df_merged3.merge(df_merged2, on='DateTime', how='inner')

Merge offgas to the other parameters.

In [ ]:
# offgas values are saved in another dataframe
## same preprocessing as the others

# methanol is zero for the whole time period and RMS is not needed
## ethanol has 30 not zero values but the highest is 0.0001 %
df_offgas.drop(['offg_ethanol', 'offg_methanol', 'offg_RMS'], axis=1, inplace=True) #
# reduce sample frequency to 1min which makes the merge easier
## mean value of each minute is taken
df_offgas.set_index('Timestamp', inplace=True)
df_resampled = df_offgas.resample('1T').mean() # could be changed to sum/mean/first/min/max...
df_resampled.reset_index(drop=False, inplace=True)
df_resampled.rename(columns={'Timestamp': 'DateTime'}, inplace=True)

# dataframe is reduced to the start and end time of the fermentation
df_offg=df_resampled[(df_resampled['DateTime'] > start_time) & (df_resampled['DateTime'] < end_time)]

## checking if sum of measurements is equal to 100
# df_offg['sum']=df_offg[['offg_co2', 'offg_n2', 'offg_o2']].sum(axis=1)
# df_offg['y_CO2']=df_offg['offg_co2']/df_offg['sum']
# df_offg['y_O2']=df_offg['offg_co2']/df_offg['sum']
# df_offg['y_N2']=df_offg['offg_co2']/df_offg['sum']

In [ ]:
# add process duration in hours as time [h]
df_merged4=df_merged3.merge(df_offg, on='DateTime', how='inner')
first_timestamp = df_merged4['DateTime'].iloc[0]  # Get the first timestamp
df_merged4['time [h]'] = (df_merged4['DateTime'] - first_timestamp) / pd.Timedelta(hours=1)
df_merged4['time [h]'] = df_merged4['time [h]'].round(2)
df_merged4.rename(columns={'Temperature Value (°C)': 'Temperature [°C]', 'Stirrer Value (rpm)': 'Stirrer [rpm]', 'pO₂ Value (pO₂)': 'DO2 [%]'}, errors='raise', inplace=True)
df_merged4.rename(columns={'offg_co2': 'Offgas CO2 [%]', 'offg_n2': 'Offgas N2 [%]', 'offg_o2': 'Offgas O2 [%]'}, errors='raise', inplace=True)
df_merged4.head(1)

Merge biomass to the other parameters.

In [ ]:
for col in df_biomass.columns:
    df_biomass[col]=df_biomass[col].str.replace(',', '.')
    df_biomass[col]=pd.to_numeric(df_biomass[col])
df_biomass.rename(columns={'hours': 'time [h]', 'biomass (g/L)': 'Biomass [g/L]'}, inplace=True)
df_biomass.head(1)

Merge HPLC data for glucose, lactic acid, acetic acid and ehtanol to other paramters.

In [ ]:
for col in df_hplc.columns:
     df_hplc[col] = df_hplc[col].astype(str).str.replace(',', '.')
     df_hplc[col] = df_hplc[col].replace('nan', float('nan'))
     df_hplc[col] = pd.to_numeric(df_hplc[col])
df_hplc.rename(columns={'Time [h]': 'time [h]'}, inplace=True)
df_hplc.head(1)

In [ ]:
# merging the biomass data with the hplc data
df_Yxs = df_biomass.merge(df_hplc, on='time [h]', how='left')

Assumption made that the parameters mu, Yxs and qs are constant for the model even though they're changing due to fermentation conditions.

In [ ]:
# calculating the growth rate 
# with 2 approaches:
## 1) from 1 to the sample points ; 2) between the two closest sample points: without log, because no exponential growth 
# df_Yxs['mu 1 [1/h]'] = (np.log(df_Yxs['Biomass [g/L]']) - np.log(df_Yxs['Biomass [g/L]'].iloc[0])) / (df_Yxs['time [h]'] - df_Yxs['time [h]'].iloc[0])
df_Yxs['mu between direct samples [1/h]'] = (abs(df_Yxs['Biomass [g/L]'].diff()) / abs(df_Yxs['time [h]'].diff()))/df_Yxs['Biomass [g/L]'].shift(1)

In [ ]:
# calculating the yield coefficient of biomass over substrate with 3 approaches: (all of them are based on the assumption that glucose is consumed, actually we dont know at all, how much is consumed)
#df_Yxs['Yxs 1.1 [gx/gs]'] = (df_Yxs['Biomass [g/L]'] - df_Yxs['Biomass [g/L]'].iloc[0]) / (df_Yxs['Glucose [g/L]'] - df_Yxs['Glucose [g/L]'].iloc[0])
#df_Yxs['Yxs 1.2 [gx/gs]'] = abs(df_Yxs['Biomass [g/L]'] - df_Yxs['Biomass [g/L]'].iloc[0]) / abs(df_Yxs['Glucose [g/L]'] - df_Yxs['Glucose [g/L]'].iloc[0])
# df_Yxs['Yxs 2.1 [gx/gs]'] = df_Yxs['Biomass [g/L]'].diff() / df_Yxs['Consumed_Glucose [g/L]'].diff()
df_Yxs['Yxs [gx/gs]'] = abs(df_Yxs['Biomass [g/L]'].diff()) / abs(df_Yxs['Consumed_Glucose [g/L]'].diff())

# 1) qs ds/dt/x = mu/Yxs
df_Yxs['qs 1 - ds/dt/x - [1/h]'] = (abs(df_Yxs['Consumed_Glucose [g/L]'].diff()) / abs(df_Yxs['time [h]'].diff()))/df_Yxs['Biomass [g/L]'].shift(1)

# 2) qs 
# df_Yxs['qs 2 - mu/Yxs -[1/h]'] = df_Yxs['mu between direct samples [1/h]']/ df_Yxs['Yxs [gx/gs]']

# Save the DataFrame to an Excel file
df_Yxs.to_excel('data/Yxs_table.xlsx', index=False)
df_Yxs.head(1)

In [ ]:
# merging the biomass, substrates and coefficients to the overall dataframe
df = df_merged4.merge(df_Yxs, on='time [h]', how='left')
df.head(3)

##### 3. Calculation of process parameters

Final dataframe created. Now, adding parameters:

In [ ]:
# the peak around the 7th hour was not fed into the batch but just to accumulate the glucose in the pipe
df.loc[df.index[428:431], 'Feed 1 Value (%)'] = 0

# after 34 h the Glucose feed was most likely empty but it was still measured as feeding
df.loc[df['time [h]'] > 34, 'Feed 1 Value (%)'] = 0
df['Feed 1 Value (%)']=df['Feed 1 Value (%)']/(10*1000)
df.rename(columns={'Feed 1 Value (%)': 'Glucose feed [L/min]'}, inplace=True)

In [ ]:
# one feed of base/acid lasts for max 5s. Since we have samples every minute I divided the value of the feed rate by 12 because I multiply by 5s*1/60s
df['Pump - Base Value (%)']=df['Pump - Base Value (%)']/(12*10*1000)
df['Pump - Acid Value (%)']=df['Pump - Acid Value (%)']/(12*10*1000)
df.rename(columns={'Pump - Base Value (%)': 'Pump - Base [L/min]', 'Pump - Acid Value (%)': 'Pump - Acid [L/min]', 'Flow - Air Value': 'Air feed [mL/min]'}, inplace=True)

# Cumulative Base Feed
df['Cum. Base feed [L/min]']=df['Pump - Base [L/min]'].cumsum()
df['Cum. Acid feed [L/min]']=df['Pump - Acid [L/min]'].cumsum()

df['Air feed [mL/min]']=df['Air feed [mL/min]']/(10)

In [ ]:
df['Offgas CO2 [mL/min]']=df['Air feed [mL/min]'] * df['Offgas CO2 [%]']/100
df['Offgas O2 [mL/min]']=df['Air feed [mL/min]'] * df['Offgas O2 [%]']/100
df['Offgas N2 [mL/min]']=df['Air feed [mL/min]'] * df['Offgas N2 [%]']/100

Volume calculation is missing evaporation term - since no weight measurements it is not included so far (no way to check if it is correct)

In [ ]:
# since the fermentation is fed-batch, the volume increases with time but sampling reduces the volume
## in total 400 mL were taken out by sampling
## in order to make it a continuous reduction it is divided by the total number of timestemps
df['Feed total [L/min]']=df['Glucose feed [L/min]']+df['Pump - Base [L/min]']+df['Pump - Acid [L/min]'] - 0.4/2767
df['Volume [L]']=df['Feed total [L/min]'].cumsum().add(1)

# # carolines euqation, question mark is just the concentration of water that goes in (last term)
# df['F_evap [kg/h]'] = (df['Air feed [mL/min]']*60/100)*(0.0235*3294) - 0.65

# Feed 1 contains 350 g/L glucose so a new parameter with the feed rate in g/h is added
df['Glucose feed [g/h]']=df['Glucose feed [L/min]']*350*60

# dissolved oxygen, stirring and the temperature fluctuate a lot so exponential smooting is applied
df['DO2 [%]- smoothed']=df['DO2 [%]'].ewm(alpha=0.1).mean()
df['Stirring [rpm]- smoothed']=df['Stirrer [rpm]'].ewm(alpha=0.1).mean()
df['T [°C]- smoothed']=df['Temperature [°C]'].ewm(alpha=0.1).mean()

In [ ]:
df['OUR [%/L/h]'] = ((df['Offgas O2 [%]']-df['Offgas O2 [%]'][0])/df['Volume [L]'])/(df['time [h]'])
df['CER [%/L/h]'] = ((df['Offgas CO2 [%]']-df['Offgas CO2 [%]'][0])/df['Volume [L]'])/(df['time [h]'])
df['RQ'] = df['CER [%/L/h]'] / df['OUR [%/L/h]']*(-1)
df.head()

In [ ]:
# the total volume in the end of the fermentation is compared with the experimental data
df['Volume [L]'].max()

In [ ]:
df.sort_index(axis=1, inplace=True)

In [ ]:
# save dataframe as csv and xlsx files
df.to_csv('data/data_combined.csv')
df.to_excel('data/data_combined.xlsx')

##### 4. Visualization

In [ ]:
df.columns

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15,8))

ax_2nd=ax.twinx()
ax_3rd=ax.twinx()
ax_4th=ax.twinx()
ax_5th=ax.twinx()

ax_5th.plot(df['time [h]'], df['Stirring [rpm]- smoothed'], label='Stirring [rpm]', color='indigo', linewidth=2)
ax_4th.plot(df['time [h]'], df['Glucose feed [g/h]'], label='Glucose feed [g/h]', color='maroon', linewidth=2)
ax_4th.scatter(df['time [h]'], df['Glucose [g/L]'], label='Glucose Conc. [g/L]', linewidth=7, color='chocolate')
ax_4th.plot(df['time [h]'], df['DO2 [%]- smoothed'], label='DO2 [%]- smoothed', color='green', linewidth=2)
ax_3rd.plot(df['time [h]'], df['T [°C]- smoothed'], label='T [°C]- smoothed', color='red', linewidth=2)
ax_3rd.set_ylim(29,30.5)
ax_2nd.plot(df['time [h]'], df['Offgas O2 [%]'], label='Offgas O2 [%]', color='deepskyblue', linewidth=2)
ax.plot(df['time [h]'], df['Volume [L]'], label='Volume [L]', color='gold', linewidth=2)
ax.plot(df['time [h]'], df['Offgas CO2 [%]'], label='Offgas CO2 [%]', color='royalblue', linewidth=2)
ax.plot(df['time [h]'], df['pH Value (pH)'], label='pH Value (pH)', color='orange', linewidth=2)
#ax.plot(df['time [h]'], df['Air feed [mL/min]'], label='Flow - Air Value', color='aqua', linewidth=2)
ax_2nd.scatter(df['time [h]'], df['Biomass [g/L]'], linewidth=7, color='plum', label='biomass')

ax_3rd.spines.right.set_position(("axes", 1.07))
ax_4th.spines.right.set_position(("axes", 1.14))
ax_5th.spines.right.set_position(("axes", 1.21))

ax.set_xlabel('time [h]')
ax.set_ylabel('pH     &     Offgas CO2 [%]     &     Volume [L]') #     Flow - Air     &, color='orange'
ax_2nd.set_ylabel('Biomass [g/L]     &     Offgas O2 [%]') #, color='royalblue'
ax_3rd.set_ylabel('T [°C]- smoothed', color='red')
ax_4th.set_ylabel('Glucose feed [g/h]     &     DO2 - smoothed&     Glucose(HPLC)[g/L] ') #, color='green'
ax_5th.set_ylabel('Stirrer [rpm]', color='indigo')

# Get the handles and labels from all the axes
handles, labels = ax.get_legend_handles_labels()
handles_2nd, labels_2nd = ax_2nd.get_legend_handles_labels()
handles_3rd, labels_3rd = ax_3rd.get_legend_handles_labels()
handles_4th, labels_4th = ax_4th.get_legend_handles_labels()
handles_5th, labels_5th = ax_5th.get_legend_handles_labels()

# Combine the handles and labels from all the axes
all_handles = handles + handles_2nd + handles_3rd + handles_4th + handles_5th
all_labels = labels + labels_2nd + labels_3rd + labels_4th + labels_5th

# Create a single legend using the combined handles and labels
ax.legend(all_handles, all_labels, loc='upper center', bbox_to_anchor=(0.5, 1.1), ncols=5)

plt.show()

In [ ]:
fig, ax = plt.subplots()
ax_2nd=ax.twinx()
#ax_3rd=ax.twinx()

ax.scatter(df['time [h]'], df['Yxs [gx/gs]'], label='Yxs [gx/gs]', color='forestgreen', linewidth=3)
ax.scatter(df['time [h]'], df['mu between direct samples [1/h]'], label='mu [1/h]', color='steelblue', linewidth=3)
ax.set_ylim(-2,1)
ax_2nd.scatter(df['time [h]'], df['Biomass [g/L]'], label='Biomass [g/L]', color='plum', linewidth=3)
ax_2nd.plot(df['time [h]'], df['Glucose feed [g/h]'], label='Glucose feed [g/h]', color='maroon', linewidth=2)
ax_2nd.scatter(df['time [h]'], df['Glucose [g/L]'], label='Glucose Conc. [g/L]', color='chocolate', linewidth=3)
#ax_3rd.spines.right.set_position(("axes", 1.15))

ax.set_xlabel('time [h]')
ax.set_ylabel('Yxs [gx/gs] & mu [1/h]')
ax_2nd.set_ylabel('Biomass [g/L] & Glucose feed [g/h] & Glucose Conc. [g/L]')
#ax_3rd.set_ylabel('Glucose feed [g/h] & Glucose Conc. [g/L]')

handles, labels = ax.get_legend_handles_labels()
handles_2nd, labels_2nd = ax_2nd.get_legend_handles_labels()
#handles_3rd, labels_3rd = ax_3rd.get_legend_handles_labels()

# Combine the handles and labels from all the axes
all_handles = handles + handles_2nd # + handles_3rd
all_labels = labels + labels_2nd # + labels_3rd

# Create a single legend using the combined handles and labels
ax.legend(all_handles, all_labels, loc='upper center', bbox_to_anchor=(0.5, 1.1), ncols=5)
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15,3))

ax[0].scatter(df['Glucose [g/L]'], df['mu between direct samples [1/h]'], label='mu [1/h]', color='steelblue', linewidth=1)
ax[1].scatter(df['Glucose [g/L]'], df['mu between direct samples [1/h]'], label='mu [1/h]', color='forestgreen', linewidth=1)

ax[0].set_xlabel('Glucose [g/L]')
ax[0].set_ylabel('mu [1/h]')
ax[1].set_xlabel('Glucose [g/L]')
ax[1].set_ylabel('mu [1/h]')


ax[0].set_title('mu 1 [1/h] vs Glucose [g/L]')
ax[1].set_title('mu 2 [1/h] vs Glucose [g/L]')
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15,3))

ax[0].scatter(df['time [h]'], df['Yxs 1.1 [gx/gs]'], label='mu [1/h]', color='steelblue', linewidth=1)
ax[1].scatter(df['time [h]'], df['Yxs 2.1 [gx/gs]'], label='Yxs [gx/gs]', color='forestgreen', linewidth=1)

#ax[0].set_ylim(0,2.5)
#ax[1].set_ylim(-3,1)

ax[0].set_title('Yxs 1.1')
ax[1].set_title('Yxs 2.1')
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(20,3))

ax[0].scatter(df['Glucose [g/L]'], df['mu 1 [1/h]'], label='mu [1/h]', color='steelblue', linewidth=1)
ax[1].scatter(df['Glucose [g/L]'], df['mu 2 [1/h]'], label='mu [1/h]', color='steelblue', linewidth=1)
#ax[1].set_ylim(-0.2, 0.2)
ax[2].scatter(df['time [h]'], df['Yxs 2.1 [gx/gs]'], label='Yxs [gx/gs]', color='forestgreen', linewidth=1)
ax[3].scatter(df['time [h]'], df['Yxs 2.2 [gx/gs]'], label='Yxs [gx/gs]', color='forestgreen', linewidth=1)
#ax[3].set_ylim(-2, 1)

ax[0].set_xlabel('Glucose [g/L]')
ax[0].set_ylabel('mu [1/h]')
ax[1].set_xlabel('Glucose [g/L]')
ax[1].set_ylabel('mu [1/h]')
ax[2].set_xlabel('time [h]')
ax[2].set_ylabel('Yxs [gx/gs]')
ax[3].set_xlabel('time [h]')
ax[3].set_ylabel('Yxs [gx/gs]')

ax[0].set_title('mu [1/h] vs Glucose [g/L]')
ax[1].set_title('mu [1/h] vs Glucose [g/L] - zoomed in')
ax[2].set_title('Yxs [gx/gs] vs time [h]')
ax[3].set_title('Yxs [gx/gs] vs time [h] - zoomed in')
plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.plot(df['time [h]'], df[['Cum. Base feed [L/min]', 'Cum. Acid feed [L/min]']], label=['Total Base [L]', 'Total Acid [L]'])

ax.set_xlabel('time [h]')
ax.set_ylabel(['Cum. Base/Acid feed [L]'])
ax.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots()

ax_sec=ax.twinx()
ax.plot(df['time [h]'], df[['CER [%/L/h]', 'OUR [%/L/h]']], label=['CER', 'OUR'])
ax_sec.plot(df['time [h]'],df['RQ'], label=['RQ'], color='green')

ax.set_xlabel('time [h]')
ax.set_ylabel(['CER [%/L/h]', 'OUR [%/L/h]'])
ax_sec.set_ylabel(['RQ'])
# Get the handles and labels from all the axes
handles, labels = ax.get_legend_handles_labels()
handles_2nd, labels_2nd = ax_sec.get_legend_handles_labels()

# Combine the handles and labels from all the axes
all_handles = handles + handles_2nd
all_labels = labels + labels_2nd

# Create a single legend using the combined handles and labels
ax.legend(all_handles, all_labels, loc='upper center', bbox_to_anchor=(0.5, 1.1), ncols=3)

plt.show()

Calculation how much glucose was fed in total when the last sample was taken:

1. Feed started as ramp at **start_feed_ramp**
2. Feed was constant starting after **start_feed_const**
3. Feed was stopped at **end_feed**

In [ ]:
start_feed_ramp = 9.92 # Start time of linear feed rate (hours)
start_feed_const = 28  # End time of linear feed rate (hours)
end_feed = 28 # Time when feed rate ends  (hours)

feed_rate_start = 0.005  # Initial feed rate (L/h)
feed_rate_end = 0.063   # Final feed rate (L/h)
feed_rate_constant = 0.102  # Constant feed rate (L/h)

feed_concentration = 350  # Concentration of feed solution (g/L)
fermentation_time = end_feed  # Total fermentation time (hours)

# Calculate the duration of ramp and constant feeding
ramp_duration = start_feed_const - start_feed_ramp
constant_duration = end_feed - start_feed_const

print(f'ramp feed for {ramp_duration} h & constant feed for {constant_duration} h')

In [ ]:
# Calculating the average feed rate based on the start and end value
avg_feedrate_ramp = (feed_rate_start + feed_rate_end) / 2 #[L/h]


# Calculating the total volume 
feed_volume_ramp = avg_feedrate_ramp * ramp_duration #[L]

# Same for constant feeding phase
feed_volume_const = feed_rate_constant * constant_duration #[L]

# Total feed volume
total_glucose = feed_volume_ramp + feed_volume_const #[L]

# Calculate the total amount of glucose in grams
total_glucose_grams = total_glucose * feed_concentration #[gs]
print("Total amount of glucose after {} hours: {:.2f} grams".format(fermentation_time, total_glucose_grams))

Calculating the amount of glc added just for the time slots inside the ramp

In [ ]:
start_feed_ramp1 = 9.92 # Start time of linear feed rate (hours)
start_feed_const1 = 21.3 # changes this value to each sample point and run it


# Calculating the average feed rate based on the start and end value
avg_feedrate_ramp = (feed_rate_start + feed_rate_end) / 2 #[L/h]

ramp_duration = start_feed_const1 - start_feed_ramp1 # [h]

# Calculating the total volume 
feed_volume_ramp = avg_feedrate_ramp * ramp_duration #[L]

# Calculate the total amount of glucose in grams
total_glucose_grams = feed_volume_ramp * feed_concentration #[gs]

print("Total amount of glucose" , total_glucose_grams)

In [ ]:
# over the whole fed-batch phase: adding 350 g/l (1l) to 1 L, --> 175 g/l  without consumption 
Yxs_consumed = (23.1 - 5.5)/(175-120.48)

print(Yxs_consumed)

Signal/noise ratio: 
Temperature

In [ ]:
# calculating the signal to noise ratio with the mean of the signal, divided by the standard deviation 

signal_amplitude  = np.mean(df['Temperature [°C]'])

temperature_signal = df['Temperature [°C]']
noise_free_portion = 30 #grad

df ['noise'] = df['Temperature [°C]'] - noise_free_portion
noise_std  = np.std(df['noise'])

snr = signal_amplitude / noise_std

df['snr2'] = df['Temperature [°C]']/ noise_std

print(signal_amplitude)
print(noise_std)
print(df['noise'])
print("SNR:", snr)
print("SNR2:",df['snr2'])

In [ ]:
fig, ax = plt.subplots()

ax.plot(df['time [h]'], df[['Temperature [°C]']], label=['Temperature]'])
ax_sec.plot(df['time [h]'],df['snr2'], label=['snr2'], color='green')

ax.set_xlabel('time [h]')
ax.set_ylabel(['Temperature [°C]]'])
ax.set_ylabel('Temperature [°C]]')
ax_sec.set_ylabel(['snr2'])

ax.legend()

plt.show()

Plot: dynamics of qs, mu and Yx/s

In [ ]:
# Create a new figure and set the size
plt.figure(figsize=(8, 6))

# Plot the first subplot
plt.subplot(3, 1, 1)  # (rows, columns, plot_number)
plt.plot(df_Yxs['time [h]'], df_Yxs['mu between direct samples [1/h]'], 'r-', label='growth rate μ  [1/h]')
plt.title('growth rate μ  [1/h]')
plt.legend()

# Add annotation for phase 1
plt.annotate('Phase 1', xy=(10, 0.6), xytext=(5, 0.8))
plt.plot([10, 10], [0.75, 0], 'k--')
# Add annotation for phase 2
plt.annotate('Phase 2', xy=(23.3, 0.3), xytext=(15, 0.8))
plt.plot([23.3, 23.3], [0.75, 0], 'k--')
plt.annotate('Phase 3', xy=(42, 0.3), xytext=(25, 0.8))

# Plot the second subplot
plt.subplot(3, 1, 2)  # (rows, columns, plot_number)
plt.plot(df_Yxs['time [h]'], df_Yxs['Yxs [gx/gs]'], 'g-', label='Yxs [x g/l / s g/l]')
plt.title('Yxs [gx/gs]')
plt.legend()

# Add annotation for phase 1
plt.annotate('Phase 1', xy=(10, 0.6), xytext=(5, 5))
plt.plot([10, 10], [6, 0], 'k--')
# Add annotation for phase 2
plt.annotate('Phase 2', xy=(23, 0.6), xytext=(15, 5))
plt.plot([23.3, 23.3], [6, 0], 'k--')
# Add annotation for phase 3
plt.annotate('Phase 3', xy=(42, 0.3),  xytext=(25, 5))

# Plot the third subplot
plt.subplot(3, 1, 3)  # (rows, columns, plot_number)
plt.plot(df_Yxs['time [h]'], df_Yxs['qs 1 - ds/dt/x - [1/h]'] , 'b-', label='qs 1 [1/h]')
plt.title('qs 1 [1/h]')
plt.legend()

# Add annotation for phase 1
plt.annotate('Phase 1', xy=(10, 0.6), xytext=(5, 0.8))
plt.plot([10, 10], [1, 0], 'k--')
# Add annotation for phase 2
plt.annotate('Phase 2', xy=(23, 0.6), xytext=(15, 0.8))
plt.plot([23.3, 23.3], [1, 0], 'k--')

plt.annotate('Phase 3', xy=(42, 0.6), xytext=(25, 0.8))

# Adjust spacing between subplots
plt.tight_layout()

# Show the plot
plt.show()